In [2]:
### Initial Setup: DO NOT RUN THIS CELL
## Terminals (order): michael, riley, kensho

import subprocess
import sys
import os
import pandas as pd
import numpy as np
import sklearn as sk
import pyarrow as pa
import gdown
from IPython.display import display, HTML

data = {
    'bank': "1dzL_SWBkBs5xrUxuGQTm04oe3USgkL9u",    # banking data
    'sales': "1QK-VgSU3AxXUw330KjYFUj8S9hzKJsG6",   # sales data
    'mcc': "1JN0bR84sgZ_o4wjKPBUmz45NeEEkVgt7",     # mcc description
}

# Download all files from Google Drive
for name, file_id in data.items():
    gdown.download(f'https://drive.google.com/uc?id={file_id}', name + '.parquet', quiet=False)
    
# Read all files and store on a dictionary of pandas dataframes
df = {}
for name in data.keys():
    df[name] = pd.read_parquet(name + '.parquet')
    
# Set pandas option to display all columns
pd.set_option('display.max_columns', None)

# Custom CSS to enable horizontal scrolling
scrolling_css = """
<style>
    .dataframe-div {
        overflow-x: auto;
        white-space: nowrap;
    }
</style>
"""

Downloading...
From: https://drive.google.com/uc?id=1dzL_SWBkBs5xrUxuGQTm04oe3USgkL9u
To: /Users/kc/Documents/VScode_projects/BrasaHack/HackBRASA/backend/bank.parquet
100%|██████████| 1.57M/1.57M [00:00<00:00, 37.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QK-VgSU3AxXUw330KjYFUj8S9hzKJsG6
To: /Users/kc/Documents/VScode_projects/BrasaHack/HackBRASA/backend/sales.parquet
100%|██████████| 6.37M/6.37M [00:00<00:00, 18.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JN0bR84sgZ_o4wjKPBUmz45NeEEkVgt7
To: /Users/kc/Documents/VScode_projects/BrasaHack/HackBRASA/backend/mcc.parquet
100%|██████████| 57.3k/57.3k [00:00<00:00, 2.03MB/s]


In [41]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

bank_df = df['bank']

# Choose a client ID and query their data
current_id = 6347736874608223396
user_data = bank_df[bank_df['document_id'] == current_id]

# Calculate inbound and outbound totals
inbound_total = user_data[user_data['type'] == 'pix_in']['value'].sum()
outbound_total = user_data[user_data['type'] == 'pix_out']['value'].sum()

# Calculate percentages
total_spending = inbound_total + outbound_total
inbound_percent = round((inbound_total / total_spending) * 100, 2) if total_spending > 0 else 0
outbound_percent = round((outbound_total / total_spending) * 100, 2) if total_spending > 0 else 0
# Create subplots
fig = make_subplots(rows=2, cols=1, specs=[[{'type':'domain'}], [{'type':'domain'}]])

# Add traces for inbound
fig.add_trace(go.Pie(
    values=[inbound_percent, 100-inbound_percent],
    labels=['', ''],
    hole=.7,
    marker_colors=['#4CAF50', '#ffffff'],
    textinfo='none',
    hoverinfo='none',
    showlegend=False
), 1, 1)

# Add traces for outbound
fig.add_trace(go.Pie(
    values=[outbound_percent, 100-outbound_percent],
    labels=['', ''],
    hole=.7,
    marker_colors=['#FF5252', '#ffffff'],
    textinfo='none',
    hoverinfo='none',
    showlegend=False
), 2, 1)

# Add annotations
fig.add_annotation(x=0.5, y=0.78, text="Inbound", showarrow=False)
fig.add_annotation(x=0.5, y=0.83, text=f"{inbound_percent}%", font=dict(size=24, color='#4CAF50'), showarrow=False)

fig.add_annotation(x=0.5, y=0.15, text="Outbound", showarrow=False)
fig.add_annotation(x=0.5, y=0.18, text=f"{outbound_percent}%", font=dict(size=24, color='#FF5252'), showarrow=False)

# Update layout
fig.update_layout(
    height=600,
    width=300,
    paper_bgcolor='#263238',
    plot_bgcolor='#263238',
    margin=dict(t=0, b=0, l=0, r=0),
    font=dict(color='white')
)

fig.show()

